# M2 axial Actuator force limit

This Jupyter notebook is to run the M2 test for the software limit for the axial actuators
An axial actuator will be issued with a force over the software limit and check that the command will be rejected by the CSCejected by the CSC.

## Import Modules

In [3]:
import asyncio
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

## Declaration of User-defined Functions

In [7]:
async def injectForce_axial_actuator(csc, actuators, force, sleep_time=5):
    """inject a force to the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    actuators : list of actuators.
    force : list of forces
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
    forces = [0.] * num_axial_actuator
    
    for idx in range(len(actuators)):
        print(f"idx: {idx}.")
        index = actuators[idx]
        forces[index] = force
        print(f"Apply the force: {abs(force)} N. to actuator: {index}")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)


## Prepare the M2 CSC and put to Enabled state

In [8]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [9]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=10)

## Do the force actuator limit Test

In [7]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

UTC time to is 2023-05-22 21:55:37.964906 now.


In [11]:
force = 450
actuators = [15]
await injectForce_axial_actuator(m2, actuators, force, sleep_time=5)

idx: 0.
Apply the force: 4900 N. to actuator: 15


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=52987816, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Max axial force (5224.83 N) >= 444.82 N.')